In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ml.models.lstm import LSTM
from ml.utils.data_utils import prepare_dataset
from argparse import Namespace

In [ ]:
# Define Configuration
args = Namespace(
    data_path='../dataset/full_dataset.csv',
    targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'],
    num_lags=10,
    forecast_steps=1,     # Single step
    test_size=0.2,
    ignore_cols=None,
    identifier='District',
    nan_constant=0,
    x_scaler='minmax',
    y_scaler='minmax',
    outlier_detection=True,
    epochs=10,
    batch_size=64,
    lr=0.001,
    hidden_size=128,
    num_layers=2,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

In [ ]:
# Prepare Data
X_train, y_train, X_test, y_test, _, _, _, _ = prepare_dataset(args)
y_train_single = y_train[:, 0, :]  # Use only t+1
y_test_single = y_test[:, 0, :]

In [ ]:
train_loader = DataLoader(TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train_single)), batch_size=args.batch_size, shuffle=True)

In [ ]:
# Model
input_dim = X_train.shape[2]
output_dim = y_train_single.shape[1]
model = LSTM(input_dim=input_dim, lstm_hidden_size=args.hidden_size,
             num_lstm_layers=args.num_layers, num_outputs=output_dim).to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
criterion = nn.MSELoss()

In [ ]:
# Train
for epoch in range(args.epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(args.device), yb.to(args.device)
        optimizer.zero_grad()
        preds = model(xb, device=args.device)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[{epoch+1}/{args.epochs}] Train Loss: {total_loss/len(train_loader):.4f}")

In [ ]:
torch.save(model.state_dict(), "basepaper_lstm.pt")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from argparse import Namespace
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from ml.models.lstm import LSTM
from ml.utils.data_utils import prepare_dataset

In [ ]:
args = Namespace(
    data_path='../dataset/full_dataset.csv',
    targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'],
    num_lags=10,
    forecast_steps=1,
    test_size=0.2,
    ignore_cols=None,
    identifier='District',
    nan_constant=0,
    x_scaler='minmax',
    y_scaler='minmax',
    outlier_detection=True,
    hidden_size=128,
    num_layers=2,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

In [ ]:
# Load Data
X_train, y_train, X_test, y_test, _, _, _, _ = prepare_dataset(args)
y_test_single = y_test[:, 0, :]  # Only t+1

# Load Model
input_dim = X_test.shape[2]
output_dim = y_test_single.shape[1]
model = LSTM(input_dim=input_dim, lstm_hidden_size=args.hidden_size,
             num_lstm_layers=args.num_layers, num_outputs=output_dim).to(args.device)
model.load_state_dict(torch.load("basepaper_lstm.pt"))
model.eval()

In [ ]:
# Predict
with torch.no_grad():
    preds = model(torch.Tensor(X_test).to(args.device), device=args.device).cpu().numpy()

In [ ]:
# Flatten and evaluate
print("\nEvaluation Metrics for All 5 Targets (t+1 only):")
for i, var in enumerate(args.targets):
    true_vals = y_test_single[:, i]
    pred_vals = preds[:, i]

    mse = mean_squared_error(true_vals, pred_vals)
    rmse = mean_squared_error(true_vals, pred_vals, squared=False)
    mae = mean_absolute_error(true_vals, pred_vals)
    r2 = r2_score(true_vals, pred_vals)
    nrmse = rmse / (true_vals.max() - true_vals.min())

    print(f"\n{var} Metrics:")
    print(f"  MSE   : {mse:.4f}")
    print(f"  RMSE  : {rmse:.4f}")
    print(f"  MAE   : {mae:.4f}")
    print(f"  R²    : {r2:.4f}")
    print(f"  NRMSE : {nrmse:.4f}")

In [ ]:
# ─────────────────────────────────────────────
# STEP 6: Visualizations

def plot_predictions(true, pred, var_name):
    plt.figure(figsize=(10, 3))
    plt.plot(true, label="Actual", alpha=0.7)
    plt.plot(pred, label="Predicted", linestyle='--')
    plt.title(f"{var_name} — Forecast for t+1")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for var in ['rb_down', 'down']:
    i = args.targets.index(var)
    plot_predictions(y_test_single[:, i], preds[:, i], var)